<a href="https://colab.research.google.com/github/KairaNithin/hate-speech-code/blob/main/muril_mean_shape_32%2C768_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -U sentence_transformers
!pip install transformers

In [ ]:
!git clone https://github.com/KairaNithin/tweets.git

Cloning into 'tweets'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 14 (delta 6), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm
from sklearn.metrics import f1_score,accuracy_score
import torch.nn as nn

In [ ]:
train_df = pd.read_csv("tweets/tweets_train.csv")
train_df = train_df.dropna().reset_index().drop('index',axis=1)
test_df = pd.read_csv("tweets/tweets_test.csv")


train_df['task_1'] = train_df['task_1'].map({'NOT':0,'HOF':1})
test_df['task_1'] = test_df['task_1'].map({'NOT':0,'HOF':1})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

In [ ]:
class create_dataset(Dataset):
    def __init__(self,data):
        self.data = data
    def __len__(self):
        return (len(self.data))
    def __getitem__(self,idx):
        encoded_inputs = tokenizer.encode_plus(self.data['tweet_raw_text'][idx],
                                                              return_tensors = 'pt',
                                                             return_attention_mask = 'True',
                                                             max_length = 64,
                                                             pad_to_max_length = True,
                                                            )
        targets = self.data['task_1'][idx]
        input_ids = torch.squeeze(encoded_inputs['input_ids'])
        attention_mask = torch.squeeze(encoded_inputs['attention_mask'])
        return{'input_ids':input_ids,
              'attention_mask':attention_mask,
               'targets':torch.tensor(targets)}

In [ ]:
train_data = create_dataset(train_df)
valid_data = create_dataset(test_df)

train_sampler = torch.utils.data.RandomSampler(train_data)
valid_sampler = torch.utils.data.RandomSampler(valid_data)

train_loader = DataLoader(train_data,
                         sampler = train_sampler,
                         batch_size = 32)
valid_loader = DataLoader(valid_data,sampler = valid_sampler,batch_size = 32)

In [ ]:
model = AutoModel.from_pretrained("google/muril-base-cased")

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for batch in train_loader:
  break
outs = model(batch['input_ids'],batch['attention_mask'])[0]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
x = torch.mean(outs,1).unsqueeze(1)
y = torch.max(outs,1)[0].unsqueeze(1)
z = torch.cat([x,y],axis=1)

In [ ]:
conv = nn.Conv1d(2,1,kernel_size=1)

In [ ]:
conv(z).squeeze(1).shape

torch.Size([32, 768])

In [ ]:
class classification_block(nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = torch.nn.Dropout(0.1)
        self.fc = nn.Linear(768,2)
    def forward(self,x):
#         x = torch.tanh(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = torch.tanh(x)
        return x

class model_mean(nn.Module):
    def __init__(self):
        super().__init__()
        self.muril_model = AutoModel.from_pretrained("google/muril-base-cased")
        self.cs = classification_block()
    def forward(self,input_ids,attention_mask):
        outs = self.muril_model(input_ids,attention_mask) 
        x = torch.mean(outs[0],1) ## shape 32,768
        y = torch.max(outs[0],1)  ## shape 32,768
        
        x = self.cs(x)
        return x

In [ ]:
model = model_mean()

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
def validate(model,data_loader):
        model.to(device)
        model.eval()
        all_outputs = []
        all_targets = []
        flat_outputs = []
        flat_targets = []
        for batch in data_loader:
            with torch.no_grad():
                outputs = model(batch['input_ids'].to(device),batch['attention_mask'].to(device))
            targets = batch['targets'].to(device)
            all_outputs.append(outputs)
            all_targets.append(targets)
        flat_outputs = [np.argmax(j.to('cpu')) for i in all_outputs for j in i]
        flat_targets = [j.to('cpu') for i in all_targets for j in i]
        return flat_outputs, flat_targets

In [ ]:
def train():
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5)
    loss_func = torch.nn.CrossEntropyLoss()
    
    epochs = 6
    score = 0
    prev_score = -1
    lr = 2e-5
    scale = 1
    
    for i in range(epochs):
        print("epoch ",i)
        print("moving forward")
        if score > prev_score:
            prev_score = score
            torch.save(model.state_dict() ,"model.pt")
        else :
            print("backtracking")
            model.load_state_dict(torch.load("model.pt"))
            lr /= (4 * scale)
            scale *= 4 
            optimizer = torch.optim.AdamW(model.parameters(), lr = lr)
            if lr < 1e-8:
                break
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)
            
            outputs = model(input_ids,attention_mask)
            
            loss = loss_func(outputs,targets)
            
            loss.backward()
            optimizer.step()
        a,b = validate(model, valid_loader)
        score = f1_score(a,b,average = 'macro')
        acc_score = accuracy_score(a,b)
        print("f1 score : ", score)
        print("acc score : ",acc_score)

In [ ]:
train()

epoch  0
moving forward


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


f1 score :  0.7615769558092265
acc score :  0.7719237435008666
epoch  1
moving forward
f1 score :  0.7750264224995532
acc score :  0.78578856152513
epoch  2
moving forward
f1 score :  0.7979115801437988
acc score :  0.8038128249566724
epoch  3
moving forward
f1 score :  0.790528556632742
acc score :  0.791681109185442
epoch  4
moving forward
backtracking
f1 score :  0.7973350035615159
acc score :  0.801733102253033
epoch  5
moving forward
backtracking
f1 score :  0.8001034671534634
acc score :  0.8038128249566724


In [ ]:
 test_en = pd.read_csv("tweets/test_en.csv")
test_hi = pd.read_csv("tweets/test_hi.csv")
test_hien = pd.read_csv("tweets/test_hien.csv")

test_en['task_1'] = test_en['task_1'].map({'NOT':0,'HOF':1})
test_hi['task_1'] = test_hi['task_1'].map({'NOT':0,'HOF':1})
test_hien['task_1'] = test_hien['task_1'].map({'NOT':0,'HOF':1})

test_en = create_dataset(test_en)
test_hi = create_dataset(test_hi)
test_hien = create_dataset(test_hien)

test_en = DataLoader(test_en,batch_size = 32)
test_hi = DataLoader(test_hi,batch_size = 32)
test_hien = DataLoader(test_hien,batch_size = 32)

In [ ]:
a,b = validate(model,test_en)
score = f1_score(a,b,average = 'macro')
acc_score = accuracy_score(a,b)
print("f1_score :",score)
print("acc_score :",acc_score)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


f1_score : 0.8729461312581517
acc_score : 0.8733153638814016


In [ ]:
a,b = validate(model,test_hi)
score = f1_score(a,b,average = 'macro')
acc_score = accuracy_score(a,b)
print("f1_score :",score)
print("acc_score :",acc_score)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


f1_score : 0.7340797635148466
acc_score : 0.7892074198988196


In [ ]:
a,b = validate(model,test_hien)
score = f1_score(a,b,average = 'macro')
acc_score = accuracy_score(a,b)
print("f1_score :",score)
print("acc_score :",acc_score)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


f1_score : 0.7736255808775592
acc_score : 0.7761290322580645
